In [2]:
import sys
import numpy as np
sys.path.append('../')
from deepod.models.time_series import DeepSVDDTS
from deepod.metrics import ts_metrics
from deepod.metrics import point_adjustment 

In [2]:
train_imu,train_audio,train_spec,val_imu,val_audio,val_spec,val_imu_normal,val_audio_normal,val_spec_normal = load_data()

train_combine = np.concatenate([np.mean(train_audio,axis=-1),train_imu],axis=1)

test_audio_all = np.concatenate([val_audio_normal,val_audio],axis=0)
test_imu_all   = np.concatenate([val_imu_normal,val_imu],axis=0)
test_combine = np.concatenate([np.mean(test_audio_all,axis=-1),test_imu_all],axis=1)

labels_all = np.concatenate([np.zeros(val_audio_normal.shape[0]),np.ones(val_audio.shape[0])])

In [9]:
clf_imu = DeepSVDDTS(epochs=50,seq_len=2)
clf_imu.fit(train_imu)
clf_audio = DeepSVDDTS(epochs=50,seq_len=2)
clf_audio.fit(np.mean(train_audio,axis=-1))
clf_all = DeepSVDDTS(epochs=50,seq_len=2)
clf_all.fit(train_combine)

477 400
Start Training...
ensemble size: 1
TSTransformerEncoder(
  (project_inp): TokenEmbedding(
    (tokenConv): Conv1d(400, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
  )
  (pos_enc): FixedPositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (conv1): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
        (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dr

/home/iot/anaconda3/envs/ai/lib/python3.8/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


epoch 20, training loss: 0.604966, time: 0.0s
epoch 30, training loss: 0.468668, time: 0.0s
epoch 40, training loss: 0.397995, time: 0.0s
epoch 50, training loss: 0.355317, time: 0.0s
Start Inference on the training data...


testing: 100%|██████████| 75/75 [00:00<00:00, 1855.73it/s]


477 8820
Start Training...
ensemble size: 1
TSTransformerEncoder(
  (project_inp): TokenEmbedding(
    (tokenConv): Conv1d(8820, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
  )
  (pos_enc): FixedPositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (conv1): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
        (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): 

testing: 100%|██████████| 75/75 [00:00<00:00, 732.63it/s]

477 9220
Start Training...
ensemble size: 1


TSTransformerEncoder(
  (project_inp): TokenEmbedding(
    (tokenConv): Conv1d(9220, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
  )
  (pos_enc): FixedPositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (conv1): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
        (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropo

testing: 100%|██████████| 75/75 [00:00<00:00, 651.61it/s]


In [10]:
scores_audio      = clf_audio.decision_function(np.mean(test_audio_all,axis=-1))
scores_imu        = clf_imu.decision_function(test_imu_all)
scores_all        = clf_all.decision_function(test_combine)

testing: 100%|██████████| 31/31 [00:00<00:00, 692.93it/s]


In [11]:
#auc, mean precision, F1, percision, recall
eval_metrics_audio = ts_metrics(labels_all, scores_audio)
eval_metrics_imu = ts_metrics(labels_all, scores_imu)
result_audio = eval_metrics_audio
result_imu = eval_metrics_imu
eval_metrics_all = ts_metrics(labels_all, scores_all)
result_all = eval_metrics_all

print(result_audio)
print(result_imu)
print(result_all)

(0.9418233699405996, 0.9202868458512083, 0.8681917213424205, 0.8676277850589778, 0.868766404199475)
(0.8665453619284431, 0.8653728918374521, 0.7944354463783238, 0.8975206611570248, 0.7125984251968503)
(0.9314369215361238, 0.9162916407675297, 0.8337787882329984, 0.8593314763231198, 0.8097112860892388)
